In [8]:
api_key = 'b6ac9f31a61869fe49b661bd084935a8'

In [ ]:
from urllib.request import urlopen
import certifi
import json
import warnings

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        response = urlopen(url, cafile=certifi.where())
        data = response.read().decode("utf-8")
    return json.loads(data)


### Get SEC tickers/cik

In [2]:
import requests

url = "https://www.sec.gov/include/ticker.txt"
local_filename = "ticker.txt"

response = requests.get(url)

if response.status_code == 200:
    with open(local_filename, "wb") as file:
        file.write(response.content)
    print(f"File '{local_filename}' downloaded successfully.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")


File 'ticker.txt' downloaded successfully.


### Make it into dataframe

In [3]:
import pandas as pd

file_name = "ticker.txt"

try:
    with open(file_name, "r") as file:
        lines = file.readlines()
        
    data = [line.strip().split('\t') for line in lines]

    # Create a DataFrame
    df = pd.DataFrame(data, columns=["Company", "CIK number"])

    print(df)
except FileNotFoundError:
    print(f"File '{file_name}' not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


      Company CIK number
0        aapl     320193
1        msft     789019
2       brk-b    1067983
3         unh     731766
4         jnj     200406
...       ...        ...
12079   hcicu    1829455
12080   hcicw    1829455
12081   hawlm      46207
12082   hbanm      49196
12083   hbanp      49196

[12084 rows x 2 columns]


### Collect Revenue Data

In [6]:
companies = df['Company']

In [9]:
import time
from tqdm import tqdm

total = 0
start = 0

# Use tqdm to add a progress bar to enumerate(companies)
for row, name in tqdm(enumerate(companies), total=len(companies), desc="Processing Companies"):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{name.upper()}?apikey={api_key}"
    json_ = get_jsonparsed_data(url)
    for _, filing in enumerate(json_):
        df.loc[row, filing['date'].split('-')[0].strip()] = filing['revenue']
    total += 1
    if total == 298:
        end = time.time()
        if int(end - start) < 60:
            time.sleep(60 - (end - start) +5)
        start = time.time()
        total = 0
        df.to_csv('revenue_2.csv')


Processing Companies: 100%|█████████████| 12084/12084 [3:51:14<00:00,  1.15s/it]


### Collect R&D

In [16]:
df = pd.DataFrame(data, columns=["Company", "CIK number"])

In [18]:
import time
from tqdm import tqdm

total = 0
start = 0

# Use tqdm to add a progress bar to enumerate(companies)
for row, name in tqdm(enumerate(companies), total=len(companies), desc="Processing Companies"):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{name.upper()}?apikey={api_key}"
    json_ = get_jsonparsed_data(url)
    for _, filing in enumerate(json_):
        df.loc[row, filing['date'].split('-')[0].strip()] = filing['researchAndDevelopmentExpenses']
    total += 1
    if total == 298:
        end = time.time()
        if int(end - start) < 60:
            time.sleep(60 - (end - start) +5)
        start = time.time()
        total = 0
        df.to_csv('r&d.csv')


Processing Companies: 100%|█████████████| 12084/12084 [4:01:16<00:00,  1.20s/it]
